In [0]:
!pip install optunity
!pip install GPy
!pip install sobol
!pip install sobol_seq

In [0]:
import optunity
import optunity.metrics
import numpy as np
import math
import pprint
import GPy
import sobol
import scipy
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, precision=6)

In [0]:
upperLimits = np.array([10, 1000, 10, 10000, 9])
lowerLimits = np.array([1,10,1,10,3])

In [0]:
parameters = []
for i in range(10):
    tp = sobol.i4_sobol(5,i)[0] * (upperLimits - lowerLimits) + lowerLimits
    parameters.append(tp)
parameters = np.array(parameters)

Measured values with mistake in the utility functions

CL | CF | FCL | Neurons | Kernel | Acc | Function
-- | -- 
3 | 200 | 4 | 1000 | 1 | 0.5172 | orig
3 | 200 | 4 | 1000 | 3 | 0.6165 | orig
3 | 200 | 4 | 1000 | 5 | 0.4363 | orig
3 | 256 | 4 | 1000 | 7 | 0.4170 | orig
3 | 200 | 4 | 1000 | 9 | 0.4471 | orig
1 | 256 | 5 | 1000 | 3 | 0.4861 | orig
2 | 256 | 5 | 500 | 3 | 0.4281 | orig
4 | 128 | 3 | 1500 | 3 | 0.7133 | orig
2 | 512 | 2 | 800 | 3 | 0.3588 | orig
2 | 128 | 2 | 2000 | 3 |  0.3727 | orig
3 | 355 | 2 | 4000 | 7 | 0.3398 | EI
6 | 10 | 4 | 1442 | 1 | 0.29 - 0.44 | HA
6 | 10 | 4 | 1442 | 3 | 0.5324 | orig
3 | 572 | 3 | 2000 | 5 | 0.4325 | ~VAR
3 | 300 | 3 | 300 | 5 | 0.5060 | EI
3 | 300 | 4 | 100 | 5 | 0.4749 | orig
2 | 400 | 3 | 50 | 7 | 0.4597 | orig
3 | 214 | 4 |  1073 | 5 | 0.4347 | -ei
4 | 417 | 5 | 64 | 7 | * | VAR
4 | 256 | 3 | 1500 | 3 | 0.6833 | orig
5 | 128 | 3 | 1000 | 3 |  0.7436 | orig
5 | 256 | 3 | 1500 | 3 | 0.7204 | orig
2 | 128 | 3 | 1500 | 7 | + | EI
3 | 256 | 4 | 500 | 5 | 0.4766 | HA




CL | CF | FCL | Neurons | Kernel | Acc | Function | LR
-- | -- 
4 | 260 | 3 | 985 | 7 | 0.7750| EI | 0.0001
3 | 75 | 4 | 1960 | 9 | 0.5518| Var | 0.0001
3 | 260 | 3 | 988 | 5 | 0.5641 | ~EI | 0.001
2 | 41 | 1 | 1287 | 7 | 0.3458 | Var | 0.001
3 | 256 | 3 | 984 | 7 | 0.7424 | EI | 0.0001
2 | 461 | 3 | 1679 | 9 | 0.4853 | Var | 0.0001
4 | 128 | 2 | 1002 | 5 | 0.8070 | EI | 0.0001
2 | 486 | 3 | 614 | 7 | 0.5413 | Var | 0.0001
3 | 129 | 2 | 999 | 5 | 0.7773 | EI | 0.0001
5 | 10 | 3 | 50 | 5 | 0.2266 | Var | 0.0001
4 | 72 | 2 | 1472 | 5 | 0.7789 | EI | 0.0001

In [0]:
parameters = [[3, 200, 4, 1000, 1], [3, 200, 4, 1000, 3], [3, 200, 4, 1000, 5],
              [3, 256, 4, 1000, 7], [3, 200, 4, 1000, 9], [1, 256, 5, 1000, 3],
              [2, 256, 5, 500, 3], [4, 128, 3, 1500, 3], [2, 512, 2, 800, 3],
              [2, 128, 2, 2000, 3], [3, 355, 2, 4000, 7], [6, 10, 4, 1442, 1],
              [6, 10, 4, 1442, 3],[3,572,3,2000,5], [3, 300, 4, 100, 5],
              [2, 400, 3, 50, 7], [3, 214, 4, 1073, 5], [4, 256, 3, 1500, 3],
              [5, 128, 3, 1000, 3], [4, 256, 3, 1500, 3], [5, 256, 3, 1500, 3],
              [4, 260, 3, 985, 7], [3, 75, 4, 1960, 9], [3, 260, 3, 988, 5],
              [2, 41, 1, 1287, 7], [3, 256, 3, 984, 7], [2, 461, 3, 1679, 9],
              [4, 128, 2, 1002, 5], [2, 486, 3, 614, 7], [3, 129, 2, 999, 5],
              [5, 10, 3, 50, 5]]
accs = [0.5172, 0.6165, 0.4363, 0.4170, 0.4471, 0.4861, 0.4281, 0.7133, 0.3588,
        0.3727, 0.3398, 0.3650, 0.5324,0.4325, 0.4749, 0.4597, 0.4347, 0.6833,
        0.7463, 0.6833, 0.7204, 0.7750, 0.5518, 0.5641, 0.3458, 0.7424, 0.4853,
        0.8070, 0.5413, 0.7713, 0.2266]
parameters = np.array(parameters)
accs = -1*np.array(accs).reshape((-1,1))

In [0]:
ker = GPy.kern.Matern52(5)

In [0]:
def expected_imp(model, x):
    pred, var = model.predict_noiseless(x)
    stddev = np.sqrt(var)
    fmin = np.min(model.Y)
    gamma = (fmin - pred)/stddev
    return stddev * (gamma * scipy.stats.norm.cdf(gamma) + scipy.stats.norm.pdf(gamma))

# Different utility function

# Expected improvement
def utility_ei(cvl, filt, fcl, neur, kern):
    return expected_imp(m, np.array([[cvl, filt, fcl, neur, kern]]))

# Variance of the GP. Maximizing this selects the point where the GP is "most unsure"
# and leads to exploration of the optimzation domain
def utility_var(cvl, filt, fcl, neur, kern):
    pred, var = m.predict_noiseless(np.array([[cvl, filt, fcl, neur, kern]]))
    return var

# Negative Prediction of the GP. Maximizing this minimizes the prediction.
# This might be useful as a last step, but wasn't required in the exercise.
def utility_f(cvl, filt, fcl, neur, kern):
    pred, var = m.predict_noiseless(np.array([[cvl, filt, fcl, neur, kern]]))
    return -pred

In [0]:
for utility in [utility_ei, utility_var, utility_f]:
    m = GPy.models.GPRegression(parameters, accs, ker)
    m.optimize(messages=False,max_f_eval = 10000)

    optimal_pars, info, _ = optunity.maximize(utility, num_evals=10000,
                                              cvl=[1,5], filt=[10,600],
                                              fcl=[1,5], neur=[50,2000],
                                              kern=[3,9])
    print(optimal_pars)